# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#gmaps.configure(api_key="")
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
cities_df = pd.read_csv("../output_data/cities.csv")

In [24]:
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

In [26]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
city_wind = cities_df[cities_df["Wind Speed"] <10]
city_wind.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44


In [28]:
city_temp = city_wind[(city_wind["Max Temp"] > 70) & (city_wind["Max Temp"] < 80)]
city_temp.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
44,44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99
59,59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62


In [29]:
city_cloud = city_temp[city_temp["Cloudiness"] == 0]
city_cloud = city_cloud.dropna(how='any')
city_cloud.reset_index(inplace=True)
city_cloud.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


In [30]:
hotel_df=city_cloud.copy()
del hotel_df["index"]
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
#For 5000 meters, I made an assumption that it is 5000 sq meters and calculated the radius which is 40.
hotellist = []

for i in range(len(city_cloud)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']   

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
        
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try: 
        hotellist.append(jsn['results'][0]['name'])

    except:
        hotellist.append("not found")
        
hotel_df["Hotel Name"] = hotellist
print(hottelist)


NameError: name 'hottelist' is not defined

In [ ]:
hotel_df.head()


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#narrowed_city_df=pd.DataFrame()
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
city = hotel_df["City"]



In [32]:
hotel_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas do Lago
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Novomar Pousada e Turismo Ltda
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda


In [36]:
# Add marker layer ontop of heat map

#markers = gmaps.marker_layer(locations)
#markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel:{hotel}" "{city} {country}" for hotel in hotellist])#
markers = gmaps.marker_layer(locations, info_box_content= [f"HOTEL NAME:  \n{hotel}"  for hotel in hotellist])



fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))